## 네이버 금융 개별종목 수집
* FinanceDataReader를 통해 수집했던 데이터를 네이버 증권 웹 페이지를 통해 직접 수집합니다.


### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 2020년 주요 상장종목
    * 빅히트 : https://finance.naver.com/item/main.nhn?code=352820
    * 카카오게임즈 : https://finance.naver.com/item/main.nhn?code=293490
    * SK바이오팜 : https://finance.naver.com/item/main.nhn?code=326030

## 라이브러리 로드

In [1]:
# 라이브러리 로드
import pandas as pd

## 수집할 URL 정하기

In [41]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
# item_code = "352820"
# item_name = "빅히트"

item_code = "326030"
item_name = "SK바이오팜"


# 종목 URL 만들기
url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page=1"
url

'https://finance.naver.com/item/sise_day.nhn?code=326030&page=1'

## pandas 코드 한 줄로 데이터 수집하기

In [42]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.
table = pd.read_html(url, encoding="cp949")
len(table)

2

In [43]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
temp = table[0]

In [44]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
temp.dropna()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.01.07,158500.0,1000.0,160000.0,160500.0,157500.0,338618.0
2,2021.01.06,159500.0,1000.0,161500.0,162500.0,158000.0,626403.0
3,2021.01.05,160500.0,6000.0,155000.0,162500.0,152500.0,1377082.0
4,2021.01.04,154500.0,14500.0,149500.0,157000.0,149000.0,3338448.0
5,2020.12.30,169000.0,1000.0,168000.0,169500.0,167000.0,221674.0
9,2020.12.29,168000.0,500.0,168500.0,171500.0,167000.0,166056.0
10,2020.12.28,167500.0,3500.0,172000.0,172000.0,167000.0,214254.0
11,2020.12.24,171000.0,0.0,171000.0,171500.0,168500.0,133848.0
12,2020.12.23,171000.0,1500.0,173000.0,173500.0,170000.0,131118.0
13,2020.12.22,172500.0,4500.0,175000.0,175500.0,172500.0,173246.0


## 페이지별 데이터 수집 함수 만들기

In [61]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.

def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """    
    url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
    table = pd.read_html(url, encoding="cp949")
    df_day = table[0].dropna()
    return df_day

In [65]:
# 함수가 잘 만들어졌는지 확인하기
page_no = 3
get_day_list(item_code, page_no)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.12.07,174000.0,2500.0,177500.0,177500.0,171500.0,255183.0
2,2020.12.04,176500.0,1000.0,175500.0,178000.0,175000.0,235976.0
3,2020.12.03,175500.0,2000.0,179000.0,179500.0,175000.0,231283.0
4,2020.12.02,177500.0,0.0,177500.0,180000.0,175000.0,290721.0
5,2020.12.01,177500.0,10000.0,185000.0,185500.0,177000.0,667685.0
9,2020.11.30,187500.0,1000.0,192500.0,192500.0,181500.0,2393234.0
10,2020.11.27,188500.0,9000.0,180000.0,189000.0,178500.0,911681.0
11,2020.11.26,179500.0,5000.0,175500.0,179500.0,172500.0,440231.0
12,2020.11.25,174500.0,500.0,174500.0,180000.0,170500.0,746914.0
13,2020.11.24,174000.0,3500.0,171000.0,174500.0,171000.0,411017.0


## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [85]:
import time
# web page 시작번호
page_no = 1
# 데이터를 저장할 빈 변수 선언
item_list = []

while True:
    df_day = get_day_list(item_code, page_no)
    item_list.append(df_day)
    
    page_no = page_no + 1
    time.sleep(0.1)
    
    if len(df_day) < 10:
        break
        

In [86]:
len(item_list)

13

## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [87]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.
df = pd.concat(item_list)
df.shape

(128, 7)

<img src="https://pandas.pydata.org/docs/_images/08_concat_row1.svg">

In [88]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.01.07,159000.0,500.0,160000.0,160500.0,157500.0,463610.0
2,2021.01.06,159500.0,1000.0,161500.0,162500.0,158000.0,626403.0
3,2021.01.05,160500.0,6000.0,155000.0,162500.0,152500.0,1377082.0
4,2021.01.04,154500.0,14500.0,149500.0,157000.0,149000.0,3338448.0
5,2020.12.30,169000.0,1000.0,168000.0,169500.0,167000.0,221674.0


In [89]:
df.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
4,2020.07.08,217000.0,500.0,214500.0,225000.0,198000.0,5990009.0
5,2020.07.07,216500.0,2000.0,219000.0,269500.0,210000.0,10105187.0
9,2020.07.06,214500.0,49500.0,214500.0,214500.0,193500.0,7126211.0
10,2020.07.03,165000.0,38000.0,165000.0,165000.0,165000.0,711921.0
11,2020.07.02,127000.0,29000.0,98000.0,127000.0,98000.0,698642.0


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [90]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
df["종목코드"] = item_code
df["종목명"] = item_name

In [91]:
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목명
1,2021.01.07,159000.0,500.0,160000.0,160500.0,157500.0,463610.0,326030,SK바이오팜
2,2021.01.06,159500.0,1000.0,161500.0,162500.0,158000.0,626403.0,326030,SK바이오팜
3,2021.01.05,160500.0,6000.0,155000.0,162500.0,152500.0,1377082.0,326030,SK바이오팜
4,2021.01.04,154500.0,14500.0,149500.0,157000.0,149000.0,3338448.0,326030,SK바이오팜
5,2020.12.30,169000.0,1000.0,168000.0,169500.0,167000.0,221674.0,326030,SK바이오팜


## 컬럼 순서 변경하기

In [95]:
# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.
df.columns

Index(['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량', '종목코드', '종목명'], dtype='object')

In [98]:
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols].copy()
df.head()

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,326030,SK바이오팜,2021.01.07,159000.0,500.0,160000.0,160500.0,157500.0,463610.0
2,326030,SK바이오팜,2021.01.06,159500.0,1000.0,161500.0,162500.0,158000.0,626403.0
3,326030,SK바이오팜,2021.01.05,160500.0,6000.0,155000.0,162500.0,152500.0,1377082.0
4,326030,SK바이오팜,2021.01.04,154500.0,14500.0,149500.0,157000.0,149000.0,3338448.0
5,326030,SK바이오팜,2020.12.30,169000.0,1000.0,168000.0,169500.0,167000.0,221674.0


## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [100]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인
print(df.shape)
df = df.drop_duplicates()
df.shape

(128, 9)


(128, 9)

## 기술통계값 구하기

In [101]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.
df.describe()

,종가,전일비,시가,고가,저가,거래량
count,128.000000,128.000000,128.000000,128.000000,128.000000,1.280000e+02
mean,172421.875000,4187.500000,172855.468750,176382.812500,169082.031250,7.204119e+05
std,14962.346112,6428.590176,16451.090999,17927.949013,14960.417241,1.264844e+06
min,127000.000000,0.000000,98000.000000,127000.000000,98000.000000,1.074820e+05
25%,163250.000000,1000.000000,164250.000000,168125.000000,159500.000000,2.098640e+05
50%,171000.000000,2500.000000,171250.000000,173250.000000,169000.000000,3.460490e+05
75%,181500.000000,4625.000000,181000.000000,185125.000000,177625.000000,6.728170e+05
max,217000.000000,49500.000000,219000.000000,269500.000000,210000.000000,1.010519e+07


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [105]:
1.28 * (10 ** 2)

128.0

In [104]:
1.280000e+02

128.0

In [106]:
1.010519e+07

10105190.0

In [103]:
# 거래량의 과학적 기수법 읽기
df["거래량"].describe()

count    1.280000e+02
mean     7.204119e+05
std      1.264844e+06
min      1.074820e+05
25%      2.098640e+05
50%      3.460490e+05
75%      6.728170e+05
max      1.010519e+07
Name: 거래량, dtype: float64

## 최근 날짜 구해서 파일명 만들기

In [109]:
# 날짜 column의 첫 row값 확인
date = df.iloc[0]["날짜"]
date

'2021.01.07'

In [113]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
file_name = f"{item_name}_{item_code}_{date}.csv"
file_name

'SK바이오팜_326030_2021.01.07.csv'

## 파일로 저장하기

In [117]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.
df.to_csv(file_name, index=False)

In [118]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다
pd.read_csv(file_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,326030,SK바이오팜,2021.01.07,159000.0,500.0,160000.0,160500.0,157500.0,463610.0
1,326030,SK바이오팜,2021.01.06,159500.0,1000.0,161500.0,162500.0,158000.0,626403.0
2,326030,SK바이오팜,2021.01.05,160500.0,6000.0,155000.0,162500.0,152500.0,1377082.0
3,326030,SK바이오팜,2021.01.04,154500.0,14500.0,149500.0,157000.0,149000.0,3338448.0
4,326030,SK바이오팜,2020.12.30,169000.0,1000.0,168000.0,169500.0,167000.0,221674.0
...,...,...,...,...,...,...,...,...,...
123,326030,SK바이오팜,2020.07.08,217000.0,500.0,214500.0,225000.0,198000.0,5990009.0
124,326030,SK바이오팜,2020.07.07,216500.0,2000.0,219000.0,269500.0,210000.0,10105187.0
125,326030,SK바이오팜,2020.07.06,214500.0,49500.0,214500.0,214500.0,193500.0,7126211.0
126,326030,SK바이오팜,2020.07.03,165000.0,38000.0,165000.0,165000.0,165000.0,711921.0


## 전체 과정을 하나의 함수로 만들기

In [127]:
def get_item_list(item_code, item_name):
    """
    일별 시세를 수집하는 함수
    """

    # web page 시작번호
    page_no = 1
    # 데이터를 저장할 빈 변수 선언
    item_list = []

    while True:
        df_day = get_day_list(item_code, page_no)
        item_list.append(df_day)

        page_no = page_no + 1
        time.sleep(0.1)

        if len(df_day) < 10:
            break
            
    df = pd.concat(item_list)

    df["종목코드"] = item_code
    df["종목명"] = item_name
    cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
    df = df[cols].copy()
    df = df.drop_duplicates()
    
    return df

In [125]:
# item_code = "352820"
# item_name = "빅히트"

item_code = "326030"
item_name = "SK바이오팜"

df = get_item_list(item_code, item_name)
df

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,326030,SK바이오팜,2021.01.07,158500.0,1000.0,160000.0,160500.0,157500.0,480831.0
2,326030,SK바이오팜,2021.01.06,159500.0,1000.0,161500.0,162500.0,158000.0,626403.0
3,326030,SK바이오팜,2021.01.05,160500.0,6000.0,155000.0,162500.0,152500.0,1377082.0
4,326030,SK바이오팜,2021.01.04,154500.0,14500.0,149500.0,157000.0,149000.0,3338448.0
5,326030,SK바이오팜,2020.12.30,169000.0,1000.0,168000.0,169500.0,167000.0,221674.0
...,...,...,...,...,...,...,...,...,...
4,326030,SK바이오팜,2020.07.08,217000.0,500.0,214500.0,225000.0,198000.0,5990009.0
5,326030,SK바이오팜,2020.07.07,216500.0,2000.0,219000.0,269500.0,210000.0,10105187.0
9,326030,SK바이오팜,2020.07.06,214500.0,49500.0,214500.0,214500.0,193500.0,7126211.0
10,326030,SK바이오팜,2020.07.03,165000.0,38000.0,165000.0,165000.0,165000.0,711921.0


## 내가 만든 함수의 기능과 소스코드가 궁금하다면?

In [128]:
# 도움말 보기
# get_item_list?


In [129]:
# 소스코드 보기
# get_item_list??
